# Practical 11: Advanced Cryptography Labs - Python

## Overview

This practical exercise is designed to introduce you to advanced cryptographic concepts. We will discuss symmetric ciphers, the RSA algorithm, advanced encryption standards (AES), electronic code books (ECB), and issues with randomness. You will also interact with these concepts using Python. This session will take about 1 hour to complete.

## Task

### Part 1: Symmetric Ciphers (10 minutes)

1. **Introduction to Symmetric Ciphers:**
   - Symmetric encryption uses the same key for both encryption and decryption.
   - Examples: AES, DES, 3DES.

In [ ]:
def pad(s, block_size=16):
    pad_len = block_size - len(s) % block_size
    return s + chr(pad_len) * pad_len


def unpad(s):
    return s[: -ord(s[-1])]


def xor_bytes(a, b):
    return bytes(x ^ y for x, y in zip(a, b))


def symmetric_encrypt(message, key):
    block_size = 16
    message = pad(message)
    encrypted_message = b""
    previous_block = b"\x00" * block_size
    for i in range(0, len(message), block_size):
        block = message[i : i + block_size].encode()
        block = xor_bytes(block, key)
        block = xor_bytes(block, previous_block)
        encrypted_message += block
        previous_block = block
    return encrypted_message


def symmetric_decrypt(ciphertext, key):
    block_size = 16
    decrypted_message = b""
    previous_block = b"\x00" * block_size
    for i in range(0, len(ciphertext), block_size):
        block = ciphertext[i : i + block_size]
        decrypted_block = xor_bytes(block, previous_block)
        decrypted_block = xor_bytes(decrypted_block, key)
        decrypted_message += decrypted_block
        previous_block = block
    return unpad(decrypted_message.decode())

2. **Example Usage:**
   - Encrypt and decrypt a message using a simple symmetric cipher.

In [ ]:
key = b"Sixteen byte key"
message = "This is a secret message"
ciphertext = symmetric_encrypt(message, key)
decrypted_message = symmetric_decrypt(ciphertext, key)

print(f"Symmetric Cipher - Plaintext: {message}")
print(f"Symmetric Cipher - Ciphertext: {ciphertext}")
print(f"Symmetric Cipher - Decrypted Message: {decrypted_message}")

### Part 2: RSA Algorithm (20 minutes)

1. **Introduction to RSA:**
   - RSA is an asymmetric encryption algorithm that uses a pair of keys: a public key for encryption and a private key for decryption.


In [ ]:
import random


def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a


def multiplicative_inverse(e, phi):
    d, x1, x2, y1 = 0, 0, 1, 1
    temp_phi = phi
    while e > 0:
        temp1, temp2 = temp_phi // e, temp_phi - temp_phi // e * e
        x, y = x2 - temp1 * x1, d - temp1 * y1
        temp_phi, e, x2, x1, d, y1 = e, temp2, x1, x, y1, y
    if temp_phi == 1:
        return d + phi


def is_prime(num):
    if num < 2:
        return False
    for n in range(2, int(num**0.5) + 1):
        if num % n == 0:
            return False
    return True


def generate_keypair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError("Both numbers must be prime.")
    elif p == q:
        raise ValueError("p and q cannot be equal")
    n = p * q
    phi = (p - 1) * (q - 1)
    e = random.randrange(1, phi)
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)
    d = multiplicative_inverse(e, phi)
    return ((e, n), (d, n))


def rsa_encrypt(message, public_key):
    key, n = public_key
    cipher = [(ord(char) ** key) % n for char in message]
    return cipher


def rsa_decrypt(ciphertext, private_key):
    key, n = private_key
    plain = [chr((char**key) % n) for char in ciphertext]
    return "".join(plain)

2. **Example Usage:**
   - Encrypt and decrypt a message using RSA.

In [ ]:
p = 61
q = 53
public_key, private_key = generate_keypair(p, q)
message = "This is a secret message"
ciphertext = rsa_encrypt(message, public_key)
decrypted_message = rsa_decrypt(ciphertext, private_key)

print(f"RSA - Plaintext: {message}")
print(f"RSA - Ciphertext: {ciphertext}")
print(f"RSA - Decrypted Message: {decrypted_message}")

### Part 3: Advanced Encryption Standards (AES) and Electronic Code Book (ECB) Mode (10 minutes)

1. **Introduction to AES and ECB:**
   - AES is a widely used symmetric encryption standard.
   - ECB is a mode of operation for block ciphers, but it has significant security weaknesses.

In [ ]:
def aes_ecb_encrypt(message, key):
    block_size = 16
    message = pad(message)
    encrypted_message = b""
    for i in range(0, len(message), block_size):
        block = message[i : i + block_size].encode()
        block = xor_bytes(block, key)
        encrypted_message += block
    return encrypted_message


def aes_ecb_decrypt(ciphertext, key):
    block_size = 16
    decrypted_message = b""
    for i in range(0, len(ciphertext), block_size):
        block = ciphertext[i : i + block_size]
        block = xor_bytes(block, key)
        decrypted_message += block
    return unpad(decrypted_message.decode())

2. **Example Usage:**
   - Encrypt and decrypt a message using AES in ECB mode.

In [ ]:
key = b"Sixteen byte key"
message = "This is a secret message"
ciphertext = aes_ecb_encrypt(message, key)
decrypted_message = aes_ecb_decrypt(ciphertext, key)

print(f"AES ECB - Plaintext: {message}")
print(f"AES ECB - Ciphertext: {ciphertext}")
print(f"AES ECB - Decrypted Message: {decrypted_message}")

### Part 4: Issues with Randomness (10 minutes)

1. **Introduction to Randomness:**
   - Randomness is crucial for secure cryptographic operations.
   - Poor sources of randomness can lead to predictable encryption keys and vulnerabilities.

In [ ]:
def generate_random_key(length):
    return bytes([random.randint(0, 255) for _ in range(length)])

2. **Example Usage:**
   - Generate a random key and use it for encryption.

In [ ]:
random_key = generate_random_key(16)
message = "This is a secret message"
ciphertext = symmetric_encrypt(message, random_key)
decrypted_message = symmetric_decrypt(ciphertext, random_key)

print(f"Random Key - Plaintext: {message}")
print(f"Random Key - Ciphertext: {ciphertext}")
print(f"Random Key - Decrypted Message: {decrypted_message}")

### Part 5: Exercises (10 minutes)

1. **Exercise 1: Implement CBC Mode**
   - TODO: Implement encryption and decryption functions for AES in CBC mode.
   - TODO: Write a function to demonstrate CBC mode encryption and decryption.

In [ ]:
# TODO: Solve Exercise 1

2. **Exercise 2: Hybrid Encryption Scheme**
   - TODO: Implement a hybrid encryption scheme combining RSA and AES.
   - TODO: Write a function that uses RSA to encrypt the AES key and AES to encrypt the message.

In [ ]:
# TODO: Solve Exercise 2

3. **Exercise 3: RSA Key Management**
   - TODO: Write functions to save and load RSA keys from files.
   - TODO: Demonstrate encryption and decryption using saved keys.

In [ ]:
# TODO: Solve Exercise 3

4. **Exercise 4: Analysis of ECB Mode**
   - TODO: Write a function to encrypt an image file using AES in ECB mode.
   - TODO: Analyze and visualize the security weaknesses of ECB mode with the encrypted image.

In [ ]:
# TODO: Solve Exercise 4

## Submission

- Submit your completed workbook to your GitHub repository.